# 🍎 파이썬 머신러닝 완벽 가이드 혼공

### 2019.05.19 ~ 2019.05.26 교재 07장

### 07. 군집화

#### 06. 군집화 실습 - 고객 세그먼테이션 

온라인 판매 데이터를 기반으로 고객 세그먼테이션을 군집화 기반으로 수행.

##### 데이터 세트 로딩과 데이터 클렌징

In [1]:
import pandas as pd 
import datetime 
import math 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

retail_df = pd.read_excel('data/07. Online Retail.xlsx')
retail_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [2]:
retail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


- CustomerID는 필요한 칼럼인데 Null값이 많음. 없으면 고객 세그멘테이션을 수행하지 못하므로 Null값 자체의 인덱스는 지우면 됨.
- 오류 데이터 삭제: 대표적으로 quantity나 unitprice가 0보다 작은 경우 = 반환 = invoiceNo의 앞자리가 'C'로 되어 있음. 

In [3]:
retail_df = retail_df[retail_df['Quantity'] > 0] # 오류 데이터 삭제 
retail_df = retail_df[retail_df['UnitPrice'] > 0]
retail_df = retail_df[retail_df['CustomerID'].notnull()] # null 값 삭제
print(retail_df.shape)
retail_df.isnull().sum()

(397884, 8)


InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [4]:
retail_df['Country'].value_counts()[:5]

United Kingdom    354321
Germany             9040
France              8341
EIRE                7236
Spain               2484
Name: Country, dtype: int64

영국이 대다수이므로 다른 데이터 삭제.

In [5]:
retail_df = retail_df[retail_df['Country'] == 'United Kingdom']
retail_df.shape

(354321, 8)

##### RFM 기반 데이터 가공 

recency(가장 최근 상품 구입일에서 오늘까지의 기간) / frequency(상품 구매 횟수) / monetary value(총 구매금액) 

In [6]:
# 구매 금액
retail_df['sale_amount'] = retail_df['Quantity'] * retail_df['UnitPrice']
retail_df['CustomerID'] = retail_df['CustomerID'].astype(int)

In [7]:
# top 5 주문자 
retail_df['CustomerID'].value_counts().head(5)

17841    7847
14096    5111
12748    4595
14606    2700
15311    2379
Name: CustomerID, dtype: int64

In [8]:
# top5 주문금액 가진 고객 
retail_df.groupby('CustomerID')['sale_amount'].sum().sort_values(ascending=False)[:5]

CustomerID
18102    259657.30
17450    194550.79
16446    168472.50
17511     91062.38
16029     81024.84
Name: sale_amount, dtype: float64

In [9]:
# 주문번호와 상품코드는 거의 1에 가까운 식별자 레벨로 
retail_df.groupby(['InvoiceNo','StockCode'])['InvoiceNo'].count().mean()

1.028702077315023

그러나 RFM 기반의 고객 세그멘테이션은 **고객 레벨로 주문 기간, 주문 횟수, 주문 금액 데이터**를 기반으로 세그멘테이션 수행하는 것 -> 주문번호+상품코드 기준의 데이터를 고객 기준의 RFM 데이터로 변경할 것.

In [10]:
aggregations = {
    'InvoiceDate':'max',
    'InvoiceNo' : 'count',
    'sale_amount': 'sum'
}
cust_df = retail_df.groupby('CustomerID').agg(aggregations)
cust_df = cust_df.rename(columns= {'InvoiceDate': 'Recency',
                                  'InvoiceNo':'Frequency',
                                  'sale_amount':'Monetary'})
cust_df = cust_df.reset_index()
cust_df.head(3)

,CustomerID,Recency,Frequency,Monetary
0,12346,2011-01-18 10:01:00,1,77183.60
1,12747,2011-12-07 14:34:00,103,4196.01
2,12748,2011-12-09 12:20:00,4595,33719.73


Recency는 고객이 가장 최근에 주문한 날짜 기반. 오늘 날짜를 기준으로 가장 최근 주문 일자를 뺀 날짜. 온라인 판매 데이터가 2010년 12월 1일에서 2011년 12월 9일까지의 데이터이므로 오늘 날짜는 2011년 12월 10일로!

In [11]:
import datetime as dt

cust_df['Recency'] = dt.datetime(2011,12,10) - cust_df['Recency']
print(cust_df.head())
cust_df['Recency'] = cust_df['Recency'].apply(lambda x: x.days+1)
print('cust_df 로우와 컬럼 건수는 ',cust_df.shape)
cust_df.head(3)

   CustomerID           Recency  Frequency  Monetary
0       12346 325 days 13:59:00          1  77183.60
1       12747   2 days 09:26:00        103   4196.01
2       12748   0 days 11:40:00       4595  33719.73
3       12749   3 days 14:04:00        199   4090.88
4       12820   3 days 08:48:00         59    942.34
cust_df 로우와 컬럼 건수는  (3920, 4)


,CustomerID,Recency,Frequency,Monetary
0,12346,326,1,77183.60
1,12747,3,103,4196.01
2,12748,1,4595,33719.73
